In [209]:
import os, sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import git

In [210]:
repo_path = git.Repo('.', search_parent_directories=True).working_tree_dir

data_folder = os.path.join(repo_path, 'data','raw', 'Beijing_AirQuality')

In [211]:
# list the files, read them and concatenate them
files = os.listdir(data_folder)
dfs = []
for file in files:
    df = pd.read_csv(os.path.join(data_folder, file), encoding='utf-8', parse_dates=['year'])
    dfs.append(df)
df = pd.concat(dfs)

In [212]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013-01-01,3,1,0,3.0,6.0,3.0,8.0,300.0,44.0,-0.9,1025.8,-20.5,0.0,NW,9.3,Shunyi
1,2,2013-01-01,3,1,1,12.0,12.0,3.0,7.0,300.0,47.0,-1.1,1026.1,-21.3,0.0,NW,9.4,Shunyi
2,3,2013-01-01,3,1,2,14.0,14.0,NaN,7.0,200.0,22.0,-1.7,1026.2,-23.0,0.0,NW,8.6,Shunyi
3,4,2013-01-01,3,1,3,12.0,12.0,3.0,5.0,NaN,NaN,-2.1,1027.3,-23.3,0.0,NW,6.6,Shunyi
4,5,2013-01-01,3,1,4,12.0,12.0,3.0,NaN,200.0,11.0,-2.4,1027.7,-22.9,0.0,NW,4.5,Shunyi


In [213]:
# drop 'No' column
if 'No' in df.columns:
    df.drop(columns='No', inplace=True)
print(df.shape)
df.head()

(420768, 17)


,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,2013-01-01,3,1,0,3.0,6.0,3.0,8.0,300.0,44.0,-0.9,1025.8,-20.5,0.0,NW,9.3,Shunyi
1,2013-01-01,3,1,1,12.0,12.0,3.0,7.0,300.0,47.0,-1.1,1026.1,-21.3,0.0,NW,9.4,Shunyi
2,2013-01-01,3,1,2,14.0,14.0,NaN,7.0,200.0,22.0,-1.7,1026.2,-23.0,0.0,NW,8.6,Shunyi
3,2013-01-01,3,1,3,12.0,12.0,3.0,5.0,NaN,NaN,-2.1,1027.3,-23.3,0.0,NW,6.6,Shunyi
4,2013-01-01,3,1,4,12.0,12.0,3.0,NaN,200.0,11.0,-2.4,1027.7,-22.9,0.0,NW,4.5,Shunyi


In [214]:
# find which columns have missing values and how many
df.isnull().sum()


year           0
month          0
day            0
hour           0
PM2.5       8739
PM10        6449
SO2         9021
NO2        12116
CO         20701
O3         13277
TEMP         398
PRES         393
DEWP         403
RAIN         390
wd          1822
WSPM         318
station        0
dtype: int64

In [215]:
# bbfill the missing values
df.ffill(inplace=True)

In [216]:
#df.isnull().sum()

In [217]:
# encode station and wd columns and then find correlation with PM2.5
df['station'] = df['station'].astype('category').cat.codes
df['wd'] = df['wd'].astype('category').cat.codes

df.corr()['PM2.5'].sort_values(ascending=False)

PM2.5      1.000000
PM10       0.876213
CO         0.773015
NO2        0.654227
SO2        0.469207
DEWP       0.112820
station    0.029748
PRES       0.019145
hour       0.010579
month      0.007143
day        0.004421
RAIN      -0.014153
year      -0.026496
wd        -0.077125
TEMP      -0.130454
O3        -0.144707
WSPM      -0.269565
Name: PM2.5, dtype: float64

In [204]:
df.head()

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,2013-01-01,3,1,0,3.0,6.0,3.0,8.0,300.0,44.0,-0.9,1025.8,-20.5,0.0,NW,9.3,Shunyi
1,2013-01-01,3,1,1,12.0,12.0,3.0,7.0,300.0,47.0,-1.1,1026.1,-21.3,0.0,NW,9.4,Shunyi
2,2013-01-01,3,1,2,14.0,14.0,3.0,7.0,200.0,22.0,-1.7,1026.2,-23.0,0.0,NW,8.6,Shunyi
3,2013-01-01,3,1,3,12.0,12.0,3.0,5.0,200.0,22.0,-2.1,1027.3,-23.3,0.0,NW,6.6,Shunyi
4,2013-01-01,3,1,4,12.0,12.0,3.0,5.0,200.0,11.0,-2.4,1027.7,-22.9,0.0,NW,4.5,Shunyi


In [205]:
# drop extra infromation from year column
df['year'] = df['year'].dt.year

In [206]:
# now combine year, month, day, hour columns to create a datetime column
df['datetime'] = pd.to_datetime(df[['year', 'month', 'day', 'hour']])

# drop year, month, day, hour columns
df.drop(columns=['year', 'month', 'day', 'hour'], inplace=True)

# set datetime as index
df.set_index('datetime', inplace=True)

# drop wind direction column
df.drop(columns='wd', inplace=True)

In [207]:
df.head()

,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,WSPM,station
datetime,,,,,,,,,,,,
2013-03-01 00:00:00,3.0,6.0,3.0,8.0,300.0,44.0,-0.9,1025.8,-20.5,0.0,9.3,Shunyi
2013-03-01 01:00:00,12.0,12.0,3.0,7.0,300.0,47.0,-1.1,1026.1,-21.3,0.0,9.4,Shunyi
2013-03-01 02:00:00,14.0,14.0,3.0,7.0,200.0,22.0,-1.7,1026.2,-23.0,0.0,8.6,Shunyi
2013-03-01 03:00:00,12.0,12.0,3.0,5.0,200.0,22.0,-2.1,1027.3,-23.3,0.0,6.6,Shunyi
2013-03-01 04:00:00,12.0,12.0,3.0,5.0,200.0,11.0,-2.4,1027.7,-22.9,0.0,4.5,Shunyi


In [179]:
df['station'].unique()

array(['Shunyi', 'Dongsi', 'Dingling', 'Wanshouxigong', 'Huairou',
       'Wanliu', 'Aotizhongxin', 'Gucheng', 'Nongzhanguan', 'Tiantan',
       'Guanyuan', 'Changping'], dtype=object)

In [150]:
# now take daily average of all columns
df_daily = df.resample('D', on='datetime').mean()

In [151]:
df_daily

,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
datetime,,,,,,,,,,,,,,,,,
2013-03-01,2013.0,3.0,1.0,11.5,7.326389,12.201389,9.184028,18.840278,404.861111,68.253472,1.173958,1026.569792,-20.148611,0.0,4.708333,2.917708,5.5
2013-03-02,2013.0,3.0,2.0,11.5,31.475694,40.517361,31.740517,56.397513,939.194444,32.621946,0.260417,1026.552778,-16.271875,0.0,6.725694,1.145139,5.5
2013-03-03,2013.0,3.0,3.0,11.5,79.291667,111.104167,49.284722,75.336806,1770.621528,22.788194,5.257292,1014.215278,-12.325347,0.0,7.232639,1.463194,5.5
2013-03-04,2013.0,3.0,4.0,11.5,21.406250,40.364583,18.761400,41.090692,725.645833,56.833333,9.610764,1017.263194,-12.773264,0.0,6.875000,2.227083,5.5
2013-03-05,2013.0,3.0,5.0,11.5,124.798611,159.236111,70.565972,105.597222,2021.958333,79.527778,6.630556,1010.551389,-7.916319,0.0,5.114583,0.992014,5.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2017-02-24,2017.0,2.0,24.0,11.5,25.444444,38.652778,9.569444,44.572917,718.750000,45.729167,4.876736,1019.349306,-11.351736,0.0,7.864583,1.902083,5.5
2017-02-25,2017.0,2.0,25.0,11.5,11.375000,21.572917,5.548611,30.118056,466.319444,59.368056,7.040278,1017.160417,-10.586111,0.0,8.336806,2.051042,5.5
2017-02-26,2017.0,2.0,26.0,11.5,27.718750,45.052083,9.975694,50.395833,702.083333,42.149306,6.769097,1018.659375,-8.005556,0.0,6.954861,1.601042,5.5


In [153]:
# weekly average
df_weekly = df.resample('W', on='datetime').mean()